In [7]:
#Config
from pyspark import SparkConf, SparkContext, HiveContext
import re
import numpy as np
import pandas as pd
import datetime
from pyhive import hive
HIVE_PORT = 10000
HIVE_HOST = 'ds-hadoop-cs01p'
HIVE_USER = 'kposminin'
CONF={'hive.vectorized.execution.enabled':'true'
    ,'mapreduce.map.memory.mb':'4096'
    ,'mapreduce.map.child.java.opts':'-Xmx4g'
    ,'mapreduce.task.io.sort.mb':'1024'
    ,'mapreduce.reduce.child.java.opts':'-Xmx4g'
    ,'mapreduce.reduce.memory.mb':'7000'
    ,'mapreduce.reduce.shuffle.input.buffer.percent':'0.5'
    ,'mapreduce.input.fileinputformat.split.minsize':'536870912'
    ,'mapreduce.input.fileinputformat.split.maxsize':'1073741824'
    ,'hive.optimize.ppd':'true'
    ,'hive.merge.smallfiles.avgsize':'536870912'
    ,'hive.merge.mapredfiles':'true'
    ,'hive.merge.mapfiles':'true'
    ,'hive.hadoop.supports.splittable.combineinputformat':'true'
    ,'hive.exec.reducers.bytes.per.reducer':'536870912'
    ,'hive.exec.parallel':'true'
    ,'hive.exec.max.created.files':'10000000'
    ,'hive.exec.compress.output':'true'
    ,'hive.exec.dynamic.partition.mode':'nonstrict'
    ,'hive.exec.max.dynamic.partitions':'1000000'
    ,'hive.exec.max.dynamic.partitions.pernode':'100000'
    ,'io.seqfile.compression.type':'BLOCK'
          }
#sc.stop()
#conf = SparkConf().set("spark.executor.instances", 32).set("spark.driver.maxResultSize", "8g")
#sc = SparkContext(conf=conf)
# hc = HiveContext(sc)

conn = hive.Connection(host=HIVE_HOST, port=HIVE_PORT, username=HIVE_USER, configuration=CONF)
cursor = conn.cursor()

In [13]:


calc_up_day = '''
insert into prod_features_liveinternet.urlfr_scores partition (ymd, target) 
select    
    urlfr,
    positive,
    total,
    log((positive + 0.1)/(total - positive + 0.1)) as score,
    ymd,
    'tinkoff_platinum_complete_application@tinkoff_actions' as target
from (
       select 
              v.ymd,
              v.urlfr,
              count(distinct v.id) as total,
              count(distinct ta.id) as positive
       from
            (select id,urlfr,ymd from prod_features_liveinternet.visits where ymd = '#day') v
            left join (select id, ymd from prod_features_liveinternet.tinkoff_actions where ymd = '#day' and action_type = 'tinkoff_platinum_complete_application') ta 
            on v.id = ta.id and v.ymd=ta.ymd
       where not urlfr like 'tinkoff.ru%'
       group by v.ymd, v.urlfr
     ) a
where
       total > 30000
       OR (positive > 1 and total > 20)
'''

day = datetime.date(2016,8,30)

for _ in range(29):
    print(day)
    cursor.execute(calc_up_day.replace('#day',str(day)))
    day += datetime.timedelta(1)
           


2016-08-30
2016-08-31
2016-09-01
2016-09-02
2016-09-03
2016-09-04
2016-09-05
2016-09-06
2016-09-07
2016-09-08
2016-09-09
2016-09-10
2016-09-11
2016-09-12
2016-09-13
2016-09-14
2016-09-15
2016-09-16
2016-09-17
2016-09-18
2016-09-19
2016-09-20
2016-09-21
2016-09-22
2016-09-23
2016-09-24
2016-09-25
2016-09-26
2016-09-27


In [6]:
print(calc_up_day.replace('#day',str(day)))


insert into prod_features_liveinternet.urlfr 
select 
    ymd,
    urlfr,
    total,
    positive,
    log((positive + 0.1)/(total - positive + 0.1)) as score
from (
       select 
              v.ymd,
              v.urlfr,
              count(distinct v.id) as total,
              count(distinct ta.id) as positive
       from
            (select id,urlfr,ymd from prod_features_liveinternet.visits where ymd = '2016-08-30') v
            left join (select id, ymd from prod_features_liveinternet.tinkoff_actions where ymd = '2016-08-30' and action_type = 'tinkoff_platinum_complete_application') ta 
            on v.id = ta.id and v.ymd=ta.ymd
       where not urlfr like 'tinkoff.ru%'
       group by v.ymd, v.urlfr
     ) a
where
       total > 30000
       OR (positive > 1 and total > 20)



In [11]:
cursor.execute('select 1')

In [12]:
cursor.fetchall()

[[1]]